In [ ]:
!wget https://github.com/bnsreenu/python_for_image_processing_APEER/blob/master/tutorial120_applying_trained_unet_model_to_large_images.ipynb

--2021-08-25 05:14:03--  https://github.com/bnsreenu/python_for_image_processing_APEER/blob/master/tutorial120_applying_trained_unet_model_to_large_images.ipynb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘tutorial120_applying_trained_unet_model_to_large_images.ipynb’

tutorial120_applyin     [ <=>                ] 125.87K  --.-KB/s    in 0.04s   

2021-08-25 05:14:03 (3.16 MB/s) - ‘tutorial120_applying_trained_unet_model_to_large_images.ipynb’ saved [128891]



In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import datasets
from scipy import stats 
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [ ]:
print(datasets.load_boston()['DESCR'])

In [ ]:
df = pd.DataFrame(datasets.load_boston()['data'])
df.columns = datasets.load_boston()['feature_names']
df['target'] = datasets.load_boston()['target']
df.describe()

In [ ]:
#split data
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target',axis=1), df.target, test_size=0.2, random_state=42)

#do modeling
model = LinearRegression()
fit_model = model.fit(X_train, y_train)
predictions = fit_model.predict(X_test)

In [ ]:
predictions

In [ ]:
predictions[0]

28.996723619824934

# Custom Method for prediction with confidence interval

In [ ]:
def get_prediction_interval(prediction, y_test, test_predictions, pi=.90):

  '''
    Get a prediction interval for a linear regression.
    
    INPUTS: 
        - Single prediction, 
        - y_test
        - All test set predictions,
        - Prediction interval threshold (default = .95) 
    OUTPUT: 
        - Prediction interval for single prediction
    '''
  #get standard deviation of y_test
  sum_errs = np.sum((y_test - test_predictions)**2)
  stdev = np.sqrt(1 / (len(y_test) - 2) * sum_errs)

  #get interval from standard deviation
  one_minus_pi = 1 - pi
  ppf_lookup = 1 - (one_minus_pi / 2)
  z_score = stats.norm.ppf(ppf_lookup)
  interval = z_score * stdev

  #generate prediction interval lower and upper bound
  lower, upper = prediction - interval, prediction + interval

  return pi, lower, prediction, upper



pi, lower_bound, predicted_value, upper_bound = get_prediction_interval(predictions[0], y_test, predictions)
print('True value(predicted value) is %.2f and %.1f percent likelihood that the true value is between %.2f and %.2f' % (predicted_value,(pi*100),lower_bound, upper_bound))

True value(predicted value) is 29.00 and 90.0 percent likelihood that the true value is between 20.81 and 37.18
